In [51]:
from sklearn.preprocessing import LabelEncoder
! pip install scikit-optimize


Defaulting to user installation because normal site-packages is not writeable


In [52]:
import pandas as pd

from skopt import BayesSearchCV
from skopt.space import Integer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score

In [53]:
data = pd.read_csv("chronic_desease.csv")

In [54]:
data

,Name,Age,Gender,BMI,Smoker,ChronicDisease,RiskLevel,InsuranceCost
0,Person_1,56,Male,33.0,Yes,1,High,36268.0
1,Person_2,46,Female,27.8,No,1,Medium,28618.0
2,Person_3,32,Female,24.0,Yes,0,Low,19763.0
3,Person_4,60,Female,39.9,No,0,Low,18311.0
4,Person_5,25,Male,27.4,No,0,Low,11893.0
...,...,...,...,...,...,...,...,...
995,Person_996,22,Male,20.4,No,0,Low,9738.0
996,Person_997,40,Male,27.3,Yes,0,Low,22997.0
997,Person_998,27,Female,18.9,No,1,Low,24105.0
998,Person_999,61,Male,34.3,No,0,Low,20626.0


In [55]:
le = LabelEncoder()
cols = ['Age', 'Gender', 'Smoker', 'RiskLevel']

for col in cols:
    data[col] = le.fit_transform(data[col])

In [56]:
X = data.drop(['Name',"RiskLevel"],axis=1)
y = data['RiskLevel']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
model = RandomForestRegressor()

In [59]:
search_space = {
    'n_estimators': Integer(50, 300),
    'max_depth': Integer(3, 20),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5)
}

In [60]:
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=search_space,
    n_iter=20,
    cv=3,
    scoring='r2',
    random_state=42,
    return_train_score=True,
)

In [61]:
bayes_search.fit(X_train, y_train)

,estimator,RandomForestRegressor()
,search_spaces,"{'max_depth': Integer(low=3...m='normalize'), 'min_samples_leaf': Integer(low=1...m='normalize'), 'min_samples_split': Integer(low=2...m='normalize'), 'n_estimators': Integer(low=5...m='normalize')}"
,optimizer_kwargs,None
,n_iter,20
,scoring,'r2'
,fit_params,None
,n_jobs,1
,n_points,1
,iid,'deprecated'
,refit,True
,cv,3


In [62]:
bayes_search.best_params_, bayes_search.best_score_

(OrderedDict([('max_depth', 10),
              ('min_samples_leaf', 4),
              ('min_samples_split', 9),
              ('n_estimators', 129)]),
 np.float64(0.674057121941698))

In [63]:
model_opt = RandomForestRegressor(
    **bayes_search.best_params_,
    random_state=42
)

In [64]:
model_opt.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",129
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",9
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",4
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples 

In [65]:
y_pred = model_opt.predict(X_test)

In [66]:
round(r2_score(y_test, y_pred)*100, 2)

83.96